### Bibliotecas

In [19]:
# Instalação manual de bibliotecas
!pip install numpy matplotlib pandas seaborn
# Bibliotecas


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import math


# 1) Considere a base de dados Breast Cancer Wisconsin (Diagnostic) (baixar em http://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic). Remova a coluna ID da base de dados e separe a coluna de Diagnosis. Divida a base de dados: 70% das primeiras amostras serão o conjunto de treinamento e os restantes 30% serão usados para teste. Faça:



In [24]:
# Leitura Inicial
feature_names = ['radius', 'texture', 'perimeter', 'area', 'smoothness', 'compactness', 'concavity', 'concave_points', 'symmetry', 'fractal_dimension']
data_raw = pd.read_csv('wdbc.data', header = None, )
data_raw.head()


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


O conjunto de dados é dividido em 3 faixas:
- 2 - 11: médio (da imagem)
- 12 - 21: desvio padrão (da imagem)
- 22 - 31: pior valor (da imagem)

Observe que a coluna 1 é a saída (diagnóstico), e a coluna 0 é um número de identificação irrelevante, que será removido dos dados.
Como não conhecemos a priori a influência do desvio padrão da imagem e do pior valor do atributo no diagnóstico, não iremos segmentar os dados.

In [26]:
# Removendo atributo irrelevante
data = data_raw.loc[:,1:31]
data.head()

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [22]:
# # Descomente esse bloco caso deseje segmentar os dados

# data_mean = data.loc[:,1:10]
# print(data_mean)
# data_std = data.loc[:,11:20]
# print(data_std)
# data_largest = data.loc[:,21:30]
# print(data_largest)

# a) Com os dados de treinamento, calcule os valores de média e desvio padrão de cada atributo para cada classe. Organize os valores em uma tabela. Comente os valores observados.







# b) Visualize os dados de treinamento usando o t-SNE (pode usar o código pronto). Comente o gráfico obtido.

# c) Com os dados de treinamento, use o coeficiente de correlação para remover atributos redundantes (considere atributos redundantes atributos com coeficiente maior ou igual a 0,9).

# d) Com os dados de treinamento, use informação mútua entre os atributos restantes de entrada e o atributo de saída e selecione os 10 melhores atributos.

# e) Treine e teste os algoritmos com os atributos selecionados: Rocchio (com distância Euclidiana), kNN (com distância Euclidiana) e Naive Bayes (considere que os atributos tenham distribuição Gaussiana). Para selecionar o melhor valor de k, divida os dados de treinamento em treinamento e em validação. Explique as considerações realizadas.

# f) Mostre os resultados sobre o conjunto de teste na forma de matriz de confusão, e com as métricas Recall, Precisão e Acurácia (a classe positiva é a maligna) e compare os resultados entre os classificadores.

# g) Realize os passos e) e f), mas usando agora todos os atributos (com exceção do ID). Compare os resultados obtidos.

# h) Faça um relatório explicando as decisões escolhidas, apresentando os resultados e os códigos usados.